In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
import tensorflow as tf

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from tensorflow.keras.optimizers import Adam

In [5]:
from tensorflow.keras.applications import VGG19, VGG16

In [6]:
import cv2
import os 
import pickle

In [7]:
from imutils import paths

In [8]:
LABELS = set(["Crown and Root Rot", "Healthy Wheat", "Leaf Rust", "Wheat Loose Smut"])

In [9]:
count = 0 
for images in paths.list_images('Images/Crown and Root Rot/'):
    print(images)
    count = count + 1
    if count == 5:
        break

Images/Crown and Root Rot/00041.jpg
Images/Crown and Root Rot/00051.jpg
Images/Crown and Root Rot/00061.jpg
Images/Crown and Root Rot/00071.jpg
Images/Crown and Root Rot/00081.png


In [10]:
image = cv2.imread('Images/Crown and Root Rot/00041.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (224, 224))

In [11]:
count = 0
for images in paths.list_images('Images/Crown and Root Rot/'):
    print(images)
    count = count + 1
    if count == 5:
        break

Images/Crown and Root Rot/00041.jpg
Images/Crown and Root Rot/00051.jpg
Images/Crown and Root Rot/00061.jpg
Images/Crown and Root Rot/00071.jpg
Images/Crown and Root Rot/00081.png


In [12]:
sample_path = list(paths.list_images('Images/Crown and Root Rot/'))[0]

In [13]:
sample_path

'Images/Crown and Root Rot/00041.jpg'

In [14]:
sample_path.split('/')

['Images', 'Crown and Root Rot', '00041.jpg']

In [15]:
os.listdir()

['.git',
 '.gitattributes',
 '.ipynb_checkpoints',
 'Images',
 'README.md',
 'Wheat Disease Detection.ipynb']

In [16]:
Labels = os.listdir('Images')

In [17]:
print(Labels)

['Crown and Root Rot', 'Healthy Wheat', 'Leaf Rust', 'Wheat Loose Smut']


In [18]:
CROWN_AND_ROOT_ROT_PATH = 'Images/Crown and Root Rot/'
HEALTHY_AND_WHEAT_PATH = 'Images/Healthy Wheat/'
LEAF_RUST_PATH = 'Images/Leaf Rust/'
WHEAT_LOOSE_SMUT_PATH = 'Images/Wheat Loose Smut/'

In [19]:
from tqdm import tqdm

In [20]:
data = []
labels = []
for image_path in tqdm(paths.list_images(CROWN_AND_ROOT_ROT_PATH)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    data.append(image)
    
    label = image_path.split('/')[-2]
    
    labels.append(label)
    
for image_path in tqdm(paths.list_images(HEALTHY_AND_WHEAT_PATH)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    data.append(image)
    
    label = image_path.split('/')[-2]
    
    labels.append(label)

for image_path in tqdm(paths.list_images(LEAF_RUST_PATH)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    data.append(image)
    
    label = image_path.split('/')[-2]
    
    labels.append(label)
    
for image_path in tqdm(paths.list_images(WHEAT_LOOSE_SMUT_PATH)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    data.append(image)
    
    label = image_path.split('/')[-2]
    
    labels.append(label)
    
    

1021it [00:04, 240.75it/s]
1146it [00:22, 51.77it/s] 
1286it [01:28, 14.49it/s] 
930it [00:04, 198.17it/s]


In [21]:
labels[-1]

'Wheat Loose Smut'

In [22]:
from tensorflow.keras.layers import Input
from tensorflow.keras import Model
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [23]:
headmodel = VGG19(weights = "imagenet", include_top = False,
             input_tensor = Input(shape = (224, 224, 3)))
model = headmodel.output
model = AveragePooling2D(pool_size= (5, 5))(model)
model = Flatten(name = 'flatten')(model)
model = Dense(512, activation = 'relu')(model)
model = Dropout(0.2)(model)
model = Dense(len(Labels), activation = 'softmax')(model)

final_model = Model(inputs = headmodel.input, outputs = model)

for layer in headmodel.layers:
    layer.trainable = False

In [24]:
opt = Adam(lr = 1e-3)
final_model.compile(loss = "categorical_crossentropy", optimizer = opt,
                   metrics = ["accuracy"])

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [26]:
data = np.array(data)
labels = np.array(labels)

lb = LabelBinarizer()
labels = lb.fit_transform(labels)


In [27]:
labels[0]

array([1, 0, 0, 0])

In [28]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2,
                                                    stratify = labels, random_state = 101)

In [45]:
len(X_train)

3506

In [29]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5296307031270157230
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7009950106
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6643352997384344691
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [30]:
final_model.fit(X_train, y_train, validation_data = (X_test, y_test), 
               epochs = 5, batch_size = 64)

Train on 3506 samples, validate on 877 samples
Epoch 1/5
3506/3506 [==============================] - 25s 7ms/sample - loss: 1.7461 - accuracy: 0.6609 - val_loss: 0.7286 - val_accuracy: 0.7765
Epoch 2/5
3506/3506 [==============================] - 13s 4ms/sample - loss: 0.4631 - accuracy: 0.8437 - val_loss: 0.3863 - val_accuracy: 0.8769loss: 0.5 - ETA: 2s - loss: 0.4741 - accu - ETA: 1s - loss: 0.4779 - accu
Epoch 3/5
3506/3506 [==============================] - 14s 4ms/sample - loss: 0.2618 - accuracy: 0.9167 - val_loss: 0.2765 - val_accuracy: 0.9145
Epoch 4/5
3506/3506 [==============================] - 14s 4ms/sample - loss: 0.1705 - accuracy: 0.9469 - val_loss: 0.2394 - val_accuracy: 0.9247
Epoch 5/5
3506/3506 [==============================] - 14s 4ms/sample - loss: 0.1249 - accuracy: 0.9641 - val_loss: 0.1847 - val_accuracy: 0.9453
